## Failed - Using pydantic to validate scraped data

In [1]:
%pip install requests
%pip install lxml
%pip install pandas
%pip install watermark
%pip install pydantic
%pip install pydantic-extra-types
%pip install pycountry
%pip install word2number

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

#### Scrape the website content

In [7]:
from pydantic import (BaseModel, 
                      HttpUrl, 
                      computed_field, 
                      field_serializer)
from pydantic_extra_types.currency_code import Currency

class BookModel(BaseModel):
  # currencies: list[Currency]
  prices: list[str]
  book_titles: list[str]
  book_urls: list[HttpUrl]
  star_ratings: list[str]

  # @computed_field  
  # @property
  # def price_stats(self) -> dict[str, float]:
  #   prices = [float(price) for price in self.price]
  #  return {"min": min(prices),
  #          "max": max(prices),
  #          "mean": mean(prices)
  #        }

  @field_serializer('star_ratings')
  def serialize_star_rating(star_ratings: list[str]) -> list[int]:
    print("-------------")
    print(star_ratings)
    return [w2n.word_to_num(star_rating.replace("star-rating ","")) for star_rating in star_ratings]

In [27]:
import requests
from lxml import html
from word2number import w2n
import pandas
from typing import Union


class BookSpider:
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            }
        bookmodel = BookModel

    def get(self):
        # loop the page number to scrape
        page_num = 1
        while True:
            try:
                url = f"https://books.toscrape.com/catalogue/page-{page_num}.html"
                output_data = self._fetch_data(url=url)
                # print(output_data)
                validated_data = self._validate_book_data(output_data.to_dict(orient="records"))
                page_num +=1
                if not validated_data: # if output_data = None or False, it will stop executing the while loop
                    break
                print(validated_data)
            except Exception as e:
                break
    
    def _fetch_data(self, url):
        response = self.session.request("GET", url)
        tree = html.fromstring(response.content)
        find_quotes_with_xpath_selector = tree.xpath("//article[@class='product_pod']")
        for find_quote in find_quotes_with_xpath_selector:
            prices = find_quote.xpath("//article[@class='product_pod']//div[@class='product_price']/p[@class='price_color']/text()")
            # bug: how to break down prices -> prices & currencies            
            book_titles = find_quote.xpath("//article[@class='product_pod']/h3/a/text()")
            book_urls = find_quote.xpath("//article[@class='product_pod']/h3/a/@href")
            star_ratings = find_quote.xpath("//article[@class='product_pod']/p/@class")

            data_list = (
                    {
                        # "currencies": currencies,
                        "prices": prices, 
                        "book_titles": book_titles,
                        "book_urls": book_urls, 
                        "star_ratings": star_ratings
                    }
                )
        return pandas.DataFrame(data_list)

    def _validate_book_data(self, books_data: dict[str, Union[str, int, float]]) -> dict[str, list]:
        # validate book data with pydantic
        validated_book_data.setdefault(list).append(dict(self.bookmodel.parse_obj(book_data).model_dump())) # model_dump is to compute field
        return validated_book_data

book_spider = BookSpider()
book_spider.get()


## Computing environment

In [4]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z

Last updated: 2024-03-09T15:41:33.518091+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.22.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.75-060175-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

lxml       : 5.1.0
word2number: 1.1
pandas     : 2.2.1
requests   : 2.31.0

Last updated: Sat Mar 09 2024 15:41:33UTC

